In [ ]:
import numpy as np
import requests
import cv2
from tensorflow.keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from tensorflow.keras.models import load_model

model = load_model('drive/MyDrive/accidentCNN_97.h5')

print('Model loaded!')

Model loaded!


In [ ]:
cap = cv2.VideoCapture()
result_list = []
y_pred_result = "Not yet"

cap.open('http://1.1.1.1:18091/?action=stream') # 카메라 Streaming 서버 주소로 바꿔야 함!!

while (True):
  _, frame = cap.read()
  img = cv2.resize(frame, dsize=(224, 224))
  # img = cv2.rotate(img, cv2.ROTATE_180) # 180도 회전
  newimg = np.asarray(img)
  pixels = newimg.astype('float32')
  pixels /= 255.0
  pixels = np.expand_dims(pixels, axis=0)
  y_pred = model.predict(pixels)

  if y_pred[0][0] > y_pred[0][1]:
    result_list.append("Accident")
  else:
    result_list.append("Non Accident")

  if len(result_list) == 10:
    if result_list.count("Accident") > result_list.count("Non Accident"):
      if y_pred_result != "Accident":
        response = requests.get('http://175.117.153.244:54321/led/on');
        y_pred_result = "Accident"
      
    else:
      if y_pred_result != "Non Accident":
        response = requests.get('http://175.117.153.244:54321/led/off');
        y_pred_result = "Non Accident"

    print("Predict result : {}".format(y_pred_result))
    result_list.clear()
  
  if cv2.waitKey(1) == ord('q'):
      break
    

KeyboardInterrupt: ignored